# Import all the dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Updated Self-Attention module with relative position embeddings



```
# This is formatted as code
```

Credit for the base self-attention:
https://github.com/pbloem/former/blob/master/former/modules.py by *pbloem*

https://discuss.pytorch.org/t/my-implementation-of-self-attention/81130 by *omer_sahban*

https://pytorch.org/docs/master/_modules/torch/nn/modules/activation.html#MultiheadAttention


In [ ]:
class DeltaMultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads = 1, dropout = 0.0):
        super(DeltaMultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout = dropout

        self.w_k = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_q = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_v = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_r = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_out = nn.Linear(emb_dim * num_heads, emb_dim)

    def forward(self, queries, keys, values):

        b, t, e = keys.shape
        h = self.num_heads

        keys = keys.transpose(2, 1).unsqueeze(-1)
        keys = keys.repeat(b, t, e, h)
        queries = queries.transpose(2, 1).unsqueeze(-1)
        queries = queries.repeat(b, t, e, h)
        values = values.transpose(2, 1).unsqueeze(-1)
        values = values.repeat(b, t, e, h)

        relational_pos_embed = self.w_r(queries).view(b, t, h, e)

        dot = queries @ keys.transpose(3, 2)
        dot = dot / np.sqrt(e) + relational_pos_embed
        dot = F.softmax(dot, dim=3)

        out = dot @ values
        out = out.transpose(1,2).contiguous().view(b, t, h * e)
        out = self.w_out(out)
        return out